In [1]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Khởi tạo ImageDataGenerator cho dữ liệu huấn luyện và kiểm tra
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Chỉ cần chuẩn hóa giá trị pixel
)

test_datagen = ImageDataGenerator(
    rescale=1./255  # Chỉ rescale, không cần augmentation
)

# Đọc dữ liệu từ thư mục đã chia
train_generator = train_datagen.flow_from_directory(
    '/kaggle/input/eyesdataclassification/train',  # Thư mục chứa các thư mục con 'Drowsy' và 'Non Drowsy'
    target_size=(224, 224),  # Kích thước đầu vào
    batch_size=32,
    class_mode='binary',  # Vì là bài toán nhị phân
    # Sử dụng ảnh grayscale
)

validation_generator = test_datagen.flow_from_directory(
    '/kaggle/input/eyesdataclassification/test',  # Thư mục chứa các thư mục con 'Drowsy' và 'Non Drowsy'
    target_size=(224, 224),  # Kích thước đầu vào
    batch_size=32,
    class_mode='binary',  # Phân loại nhị phân
     # Sử dụng ảnh grayscale
)


Found 5713 images belonging to 2 classes.
Found 239 images belonging to 2 classes.


In [2]:
from sklearn.neural_network import MLPClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import classification_report, accuracy_score
import numpy as np

# --- 1. Lấy toàn bộ dữ liệu ---
# Lấy dữ liệu từ train_generator
X_train, y_train = [], []
for _ in range(len(train_generator)):
    X_batch, y_batch = next(train_generator)
    X_train.append(X_batch)
    y_train.append(y_batch)

X_train = np.concatenate(X_train)
y_train = np.concatenate(y_train)

# Lấy dữ liệu từ validation_generator
X_val, y_val = [], []
for _ in range(len(validation_generator)):
    X_batch, y_batch = next(validation_generator)
    X_val.append(X_batch)
    y_val.append(y_batch)

X_val = np.concatenate(X_val)
y_val = np.concatenate(y_val)

# --- 2. MLP không giảm chiều ---
# Chuyển dữ liệu thành dạng 1D
X_train_flattened = X_train.reshape(X_train.shape[0], -1)
X_val_flattened = X_val.reshape(X_val.shape[0], -1)

# Khởi tạo và huấn luyện MLP
mlp_no_pca = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
mlp_no_pca.fit(X_train_flattened, y_train)

# Dự đoán và đánh giá
y_pred_no_pca = mlp_no_pca.predict(X_val_flattened)
accuracy_no_pca = accuracy_score(y_val, y_pred_no_pca)
print(f"MLP Accuracy (No PCA): {accuracy_no_pca:.4f}")
print("Classification Report (No PCA):")
print(classification_report(y_val, y_pred_no_pca))


MLP Accuracy (No PCA): 0.9833
Classification Report (No PCA):
              precision    recall  f1-score   support

         0.0       0.97      0.99      0.98       108
         1.0       0.99      0.98      0.98       131

    accuracy                           0.98       239
   macro avg       0.98      0.98      0.98       239
weighted avg       0.98      0.98      0.98       239



In [3]:
from sklearn.decomposition import PCA
# --- 3. MLP giảm chiều bằng PCA ---
# Áp dụng PCA giữ lại 90% phương sai
pca = PCA(n_components=0.9, svd_solver='full')
X_train_pca = pca.fit_transform(X_train_flattened)
X_val_pca = pca.transform(X_val_flattened)

# Khởi tạo và huấn luyện MLP với dữ liệu đã giảm chiều
mlp_with_pca = MLPClassifier(hidden_layer_sizes=(128, 64), max_iter=500, random_state=42)
mlp_with_pca.fit(X_train_pca, y_train)

# Dự đoán và đánh giá
y_pred_with_pca = mlp_with_pca.predict(X_val_pca)
accuracy_with_pca = accuracy_score(y_val, y_pred_with_pca)
print(f"MLP Accuracy (With PCA): {accuracy_with_pca:.4f}")
print("Classification Report (With PCA):")
print(classification_report(y_val, y_pred_with_pca))

MLP Accuracy (With PCA): 1.0000
Classification Report (With PCA):
              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       108
         1.0       1.00      1.00      1.00       131

    accuracy                           1.00       239
   macro avg       1.00      1.00      1.00       239
weighted avg       1.00      1.00      1.00       239

